In [1]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.7/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.7/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

In [3]:
import os, time
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [4]:
import dask as dask, dask_cudf
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import subprocess

cluster = LocalCUDACluster()
client = Client(cluster)
client

/usr/local/lib/python3.7/site-packages/distributed/client.py:1140: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| tornado | 5.1.1  | 5.1.1     | 6.1     |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Client Scheduler: tcp://127.0.0.1:41987 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 12.72 GiB


In [5]:
features = [
    'text_tokens',    ###############
    'hashtags',       #Tweet Features
    'tweet_id',       #
    'media',          #
    'links',          #
    'domains',        #
    'tweet_type',     #
    'language',       #
    'timestamp',      ###############
    'a_user_id',              ###########################
    'a_follower_count',       #Engaged With User Features
    'a_following_count',      #
    'a_is_verified',          #
    'a_account_creation',     ###########################
    'b_user_id',              #######################
    'b_follower_count',       #Engaging User Features
    'b_following_count',      #
    'b_is_verified',          #
    'b_account_creation',     #######################
    'b_follows_a',    #################### Engagement Features
    'reply',          #Target Reply
    'retweet',        #Target Retweet    
    'retweet_comment',#Target Retweet with comment
    'like',           #Target Like                   ####################
]

In [10]:
%%time
df = dask_cudf.read_csv('part-*', sep='\x01', names=features)
df.head()

CPU times: user 50.1 ms, sys: 12.9 ms, total: 63 ms
Wall time: 421 ms


In [12]:
%%time
df = df.drop('text_tokens', axis=1)
df, = dask.persist(df)
_ = wait(df)

CPU times: user 986 ms, sys: 104 ms, total: 1.09 s
Wall time: 15.9 s


In [13]:
%%time
df.head()

CPU times: user 16.4 ms, sys: 829 µs, total: 17.2 ms
Wall time: 37.2 ms


,hashtags,tweet_id,media,links,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_user_id,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like
0,<NA>,5DCE3020423313CB0AC2631DEF94D2F5,<NA>,<NA>,<NA>,Retweet,488B32D24BD4BB44172EB981C1BCA6FA,1613237034,BC9980C5385A68894C2862F41884FB27,2473,662,False,1261859734,2AEB7AFFD36CA43C0C963F24F75BCCAC,169,339,False,1520886748,False,<NA>,<NA>,<NA>,<NA>
1,<NA>,2B6F5C7C4F04F34543C09DD6E58C8F66,<NA>,2886BC98B8B910E86A792E56681FC841,9EFF000CDB18B710CDDB43EE1D8C300B,TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,1613748600,C0EBA033A5C0D2AD91B4500958E2C633,4418640,228,True,1266804490,865C52D28FA84E5E233B530C6FDAECC2,393,1190,False,1237570695,False,<NA>,<NA>,<NA>,<NA>
2,<NA>,784D3AE5E6429392245BD34FF4C82B5C,<NA>,B8FEE70DF7DA75C584F9DA8C38E99874,D56FA7843AF6F2BC53A2E192B542EA58,TopLevel,B0FA488F2911701DD8EC5B1EA5E322D8,1613386238,03EA05E7440140ABF047BC216CE34917,219715,3685,True,1202617218,FE83E568BD4B2ACDBAF876CFBB6C7296,629,1473,False,1263176351,False,<NA>,<NA>,<NA>,1.613388292e+09
3,<NA>,CA49ACBB31921F2A0FAC1CCCC99541F6,<NA>,<NA>,<NA>,Retweet,B0FA488F2911701DD8EC5B1EA5E322D8,1613708640,2700B9E175757F126C082D6A5742787D,2388283,13511,True,1251645191,DB0201BF93E4CCA1B55932AC36923E43,123,200,False,1268276559,False,<NA>,<NA>,<NA>,<NA>
4,<NA>,2C95B42C651B36D84584F3653D722142,Photo,<NA>,<NA>,TopLevel,E7F038DE3EAD397AEC9193686C911677,1612586018,B7E441F2D886830FB3F0CE29CB35CB6B,414,720,False,1578273274,04F45C4A769FE2D7C792A96C92342DC2,134,379,False,1483862063,True,<NA>,<NA>,<NA>,1.612587384e+09


In [14]:
%%time
df['id'] = 1
df['id'] = df['id'].cumsum()
df['id'] = df['id'].astype('int32')

df['reply'] = df['reply'].fillna(0)
df['retweet'] = df['retweet'].fillna(0)
df['retweet_comment'] = df['retweet_comment'].fillna(0)
df['like'] = df['like'].fillna(0)

df['reply'] = df['reply'].astype('int32')
df['retweet'] = df['retweet'].astype('int32')
df['retweet_comment'] = df['retweet_comment'].astype('int32')
df['like'] = df['like'].astype('int32')
df, = dask.persist(df)
_ = wait(df)

CPU times: user 1.3 s, sys: 186 ms, total: 1.48 s
Wall time: 8.73 s


In [15]:
%%time

df['timestamp'] = df['timestamp'].astype( np.int32 )
df['a_follower_count'] = df['a_follower_count'].astype( np.int32 )
df['a_following_count'] = df['a_following_count'].astype( np.int32 )
df['a_account_creation'] = df['a_account_creation'].astype( np.int32 )
df['b_follower_count'] = df['b_follower_count'].astype( np.int32 )
df['b_following_count'] = df['b_following_count'].astype( np.int32 )
df['b_account_creation'] = df['b_account_creation'].astype( np.int32 )

df, = dask.persist(df)
_ = wait(df)

CPU times: user 324 ms, sys: 23.9 ms, total: 348 ms
Wall time: 1.98 s


In [16]:
%%time
tweet = df[['tweet_id']]
tweet = tweet.drop_duplicates(split_out=16)
tweet['tweet_encode'] = 1
tweet['tweet_encode'] = tweet['tweet_encode'].cumsum()
tweet, = dask.persist(tweet)
_ = wait(tweet)

CPU times: user 450 ms, sys: 29 ms, total: 479 ms
Wall time: 2.56 s


In [17]:
%%time
tweet.head()

CPU times: user 9.24 ms, sys: 845 µs, total: 10.1 ms
Wall time: 16.9 ms


,tweet_id,tweet_encode
312867,00009D6FFA2FBF7F12F704F00558469A,1
52249,0000EEDC406DACDC9AA9385CB6319910,2
108636,0001347C7EE50DF8A45BC288C1C6387D,3
173251,000185AEB2BA91BC9112BE9EDC7E522E,4
59354,000233E63BC10675CAD2EED950912FBF,5


In [18]:
%%time
df = df.merge(tweet,on='tweet_id',how='left')
df = df.drop('tweet_id',axis=1)
df.columns = [i if i!='tweet_encode' else 'tweet_id' for i in df.columns]
df, = dask.persist(df)
wait(df)
del tweet

CPU times: user 487 ms, sys: 51.3 ms, total: 539 ms
Wall time: 4.42 s


In [19]:
%%time
df.head()

CPU times: user 14.7 ms, sys: 1.25 ms, total: 15.9 ms
Wall time: 29.3 ms


,hashtags,media,links,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_user_id,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,id,tweet_id
0,A9A9AB99F47CD2A83E9EE7514B69A7A5\t421BABC4007F...,<NA>,<NA>,<NA>,TopLevel,9FCF19233EAD65EA6E32C2E6DC03A444,1612958459,8A4F63F151BB35932C97E3061FE28956,338,513,False,1573442953,283DF404BFD23C479F469A79CFBDB2B4,95,296,False,1588059955,True,0,1612958531,0,0,80867,78182
1,<NA>,<NA>,817BCB46F154285B56AC0E766DF2CF8B,40CFBD548620F702FF082BD81F43EB75,TopLevel,B8B04128918BBF54E2E178BFF1ABA833,1612665457,03CFBF582F540AADF17930781C81A0D7,212907,38,True,1255087101,273C8ED0F82DF8A6117CAED1262BEA85,165,149,False,1514637499,False,0,0,0,0,81184,100270
2,<NA>,Photo,<NA>,<NA>,TopLevel,E7F038DE3EAD397AEC9193686C911677,1612402594,BC855CB64AE7718F67D9B23F1C3FCF02,27425,1072,False,1270865225,7A2D6298AEF3A7091D44420DF256AE49,241,105,False,1571999901,False,0,0,0,0,44978,98143
3,<NA>,<NA>,<NA>,<NA>,Quote,488B32D24BD4BB44172EB981C1BCA6FA,1613508530,93D3C7A9E92043A56BE3A42EEBA209A3,536,485,False,1386719272,72562B245356283A06B1EF5B2893CA0A,690,756,False,1384291304,True,0,0,0,1613514494,45451,50822
4,<NA>,<NA>,<NA>,<NA>,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,1612934459,8E2016E4EDAE61EF4069384D60AD1726,7087091,319,True,1358063733,E7D670B27D8790FC863491D5AE1AFE8F,546,261,False,1577702219,False,0,0,0,0,51550,134524


In [20]:
%%time
user_a = df[['a_user_id']].drop_duplicates(split_out=16)
user_a, = dask.persist(user_a)
_ = wait(user_a)
user_b = df[['b_user_id']].drop_duplicates(split_out=16)
user_b, = dask.persist(user_b)
wait(user_b)

user_a.columns = ['user_id']
user_b.columns = ['user_id']
user_b['dummy'] = 1
user_a = user_a.merge(user_b,on='user_id',how='outer')
user_a = user_a.drop('dummy',axis=1)
user_a, = dask.persist(user_a)
wait(user_a)
del user_b

user_a['user_encode'] = 1
user_a['user_encode'] = user_a['user_encode'].cumsum()
user_a, = dask.persist(user_a)
_ = wait(user_a)

CPU times: user 1 s, sys: 80.4 ms, total: 1.08 s
Wall time: 4.11 s


In [21]:
%%time
df = df.merge(user_a,left_on='a_user_id',right_on='user_id',how='left')
df = df.drop(['a_user_id','user_id'],axis=1)
df.columns = [i if i!='user_encode' else 'a_user_id' for i in df.columns]
df, = dask.persist(df)
_ = wait(df)

CPU times: user 524 ms, sys: 46.7 ms, total: 570 ms
Wall time: 3.86 s


In [22]:
%%time
df = df.merge(user_a,left_on='b_user_id',right_on='user_id',how='left')
df = df.drop(['b_user_id','user_id'],axis=1)
df.columns = [i if i!='user_encode' else 'b_user_id' for i in df.columns]
df, = dask.persist(df)
wait(df)
del user_a

CPU times: user 519 ms, sys: 50.1 ms, total: 570 ms
Wall time: 3.76 s


In [23]:
%%time
df.head()

CPU times: user 14.8 ms, sys: 922 µs, total: 15.7 ms
Wall time: 27 ms


,hashtags,media,links,domains,tweet_type,language,timestamp,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,id,tweet_id,a_user_id,b_user_id
0,<NA>,<NA>,<NA>,<NA>,TopLevel,1F73BB863A39DB62B4A55B7E558DB1E8,1613287160,2036439,204,True,1352030774,3372,2827,False,1526668877,False,0,0,0,0,4356504,450009,123348,640
1,200655D87FE7E0A0006234DB127A378F,<NA>,F536ED867A856C4F79FE7DF2ABDF705B,37F6EF8038F7A0B9A07DF964E16EB09A,Retweet,B0FA488F2911701DD8EC5B1EA5E322D8,1613757161,1379011,511,True,1253288083,176,557,False,1483506287,False,0,0,0,0,5616316,440225,122734,63314
2,<NA>,Video,<NA>,<NA>,Retweet,B0FA488F2911701DD8EC5B1EA5E322D8,1613989481,123094,370,False,1391706825,294,350,False,1329637839,False,0,1613997099,0,0,3297052,60417,217880,65320
3,<NA>,<NA>,<NA>,<NA>,Quote,E6936751CBF4F921F7DE1AEF33A16ED0,1612913369,1685,1755,False,1384742767,25,92,False,1590276962,True,0,0,0,1612913759,1482019,597946,177067,51696
4,<NA>,Photo\tPhoto,<NA>,<NA>,Retweet,488B32D24BD4BB44172EB981C1BCA6FA,1613419546,2721,2213,False,1307938055,1668,2065,False,1366489077,True,0,0,0,0,732662,426015,141352,139552


In [26]:
%%time
def count_token(ds,token):
    not_null = ds.isnull()==0
    return ((ds.str.count(token)+1)*not_null).fillna(0)
df['len_hashtags'] = df['hashtags'].map_partitions(lambda ds: count_token(ds,'\t'))
df['len_domains'] = df['domains'].map_partitions(lambda ds: count_token(ds,'\t'))
df['len_links'] = df['links'].map_partitions(lambda ds: count_token(ds,'\t'))

df, = dask.persist(df)
_ = wait(df)

CPU times: user 1.12 s, sys: 31.4 ms, total: 1.16 s
Wall time: 1.48 s


In [27]:
%%time
df.head()

CPU times: user 16 ms, sys: 115 µs, total: 16.1 ms
Wall time: 28.9 ms


,hashtags,media,links,domains,tweet_type,language,timestamp,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,id,tweet_id,a_user_id,b_user_id,len_hashtags,len_domains,len_links
0,<NA>,<NA>,<NA>,<NA>,TopLevel,1F73BB863A39DB62B4A55B7E558DB1E8,1613287160,2036439,204,True,1352030774,3372,2827,False,1526668877,False,0,0,0,0,4356504,450009,123348,640,0,0,0
1,200655D87FE7E0A0006234DB127A378F,<NA>,F536ED867A856C4F79FE7DF2ABDF705B,37F6EF8038F7A0B9A07DF964E16EB09A,Retweet,B0FA488F2911701DD8EC5B1EA5E322D8,1613757161,1379011,511,True,1253288083,176,557,False,1483506287,False,0,0,0,0,5616316,440225,122734,63314,1,1,1
2,<NA>,Video,<NA>,<NA>,Retweet,B0FA488F2911701DD8EC5B1EA5E322D8,1613989481,123094,370,False,1391706825,294,350,False,1329637839,False,0,1613997099,0,0,3297052,60417,217880,65320,0,0,0
3,<NA>,<NA>,<NA>,<NA>,Quote,E6936751CBF4F921F7DE1AEF33A16ED0,1612913369,1685,1755,False,1384742767,25,92,False,1590276962,True,0,0,0,1612913759,1482019,597946,177067,51696,0,0,0
4,<NA>,Photo\tPhoto,<NA>,<NA>,Retweet,488B32D24BD4BB44172EB981C1BCA6FA,1613419546,2721,2213,False,1307938055,1668,2065,False,1366489077,True,0,0,0,0,732662,426015,141352,139552,0,0,0


In [28]:
def factorize_small_cardinality(df,col):
    tmp_col = f'{col}_encode'
    tmp = df[col].unique().compute()
    tmp = tmp.to_frame().reset_index()
    tmp = tmp.rename(columns={'index':tmp_col})
    df = df.merge(tmp,on=col,how='left')
    df, = dask.persist(df)
    wait(df)
    del tmp
    df = df.drop(col,axis=1)
    df, = dask.persist(df)
    wait(df)
    df.columns = [i if i!=tmp_col else col for i in df.columns ]
    return df

In [29]:
%%time
for col in ['language','tweet_type']:
    df = factorize_small_cardinality(df,col)
    df[col] = df[col].astype('int8')

CPU times: user 754 ms, sys: 36 ms, total: 790 ms
Wall time: 2.54 s


In [30]:
%%time
df.head()

CPU times: user 19.4 ms, sys: 1.5 ms, total: 20.9 ms
Wall time: 44 ms


,hashtags,media,links,domains,timestamp,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,id,tweet_id,a_user_id,b_user_id,len_hashtags,len_domains,len_links,language,tweet_type
0,<NA>,Photo,<NA>,<NA>,1613575431,304,637,False,1538731851,2625,2587,False,1449472125,True,0,0,0,1613575581,308728,2094195,123065,101538,0,0,0,61,1
1,<NA>,<NA>,<NA>,<NA>,1612886582,23310,2119,True,1246300047,120,209,False,1309436731,False,0,0,0,0,5302839,1935527,200136,7414,0,0,0,48,1
2,<NA>,<NA>,<NA>,<NA>,1614130621,38519,21564,False,1257360731,1032,1926,False,1533159537,True,0,0,0,1614164644,4558586,2178656,217822,26088,0,0,0,48,2
3,<NA>,Photo\tPhoto,<NA>,<NA>,1614184644,14106,1645,False,1223668013,265,335,False,1249692829,False,0,0,0,1614189647,5536323,1894129,150116,150980,0,0,0,19,1
4,8EC708C0073EDDE57F3BB17ECA262D9E\tCBFFDD847281...,Video,<NA>,<NA>,1613842378,2652,175,False,1562820796,406,794,False,1572957202,False,0,0,0,1613855637,1017697,1981260,127092,80160,2,0,0,45,2


In [32]:
%%time
def count_token(ds,token):
    not_null = ds.isnull()==0
    return ((ds.str.count(token))*not_null).fillna(0)

df['n_photo'] = df['media'].map_partitions(lambda ds: count_token(ds, 'Photo'))
df['n_gif'] = df['media'].map_partitions(lambda ds: count_token(ds, 'GIF'))
df['n_video'] = df['media'].map_partitions(lambda ds: count_token(ds, 'Video'))

df, = dask.persist(df)
_ = wait(df)

CPU times: user 270 ms, sys: 18.8 ms, total: 289 ms
Wall time: 945 ms


In [33]:
%%time
df.head()

CPU times: user 14.2 ms, sys: 1.62 ms, total: 15.8 ms
Wall time: 26.2 ms


,hashtags,media,links,domains,timestamp,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,id,tweet_id,a_user_id,b_user_id,len_hashtags,len_domains,len_links,language,tweet_type,n_photo,n_gif,n_video
0,<NA>,Photo,<NA>,<NA>,1613575431,304,637,False,1538731851,2625,2587,False,1449472125,True,0,0,0,1613575581,308728,2094195,123065,101538,0,0,0,61,1,1,0,0
1,<NA>,<NA>,<NA>,<NA>,1612886582,23310,2119,True,1246300047,120,209,False,1309436731,False,0,0,0,0,5302839,1935527,200136,7414,0,0,0,48,1,0,0,0
2,<NA>,<NA>,<NA>,<NA>,1614130621,38519,21564,False,1257360731,1032,1926,False,1533159537,True,0,0,0,1614164644,4558586,2178656,217822,26088,0,0,0,48,2,0,0,0
3,<NA>,Photo\tPhoto,<NA>,<NA>,1614184644,14106,1645,False,1223668013,265,335,False,1249692829,False,0,0,0,1614189647,5536323,1894129,150116,150980,0,0,0,19,1,2,0,0
4,8EC708C0073EDDE57F3BB17ECA262D9E\tCBFFDD847281...,Video,<NA>,<NA>,1613842378,2652,175,False,1562820796,406,794,False,1572957202,False,0,0,0,1613855637,1017697,1981260,127092,80160,2,0,0,45,2,0,0,1


In [34]:
df['date'] = df['timestamp'].astype('datetime64[s]')
df['hour'] = df['date'].dt.hour
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek
df = df.drop('date', axis=1)

In [35]:
df.head()

,hashtags,media,links,domains,timestamp,a_follower_count,a_following_count,a_is_verified,a_account_creation,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,id,tweet_id,a_user_id,b_user_id,len_hashtags,len_domains,len_links,language,tweet_type,n_photo,n_gif,n_video,hour,day,month,day_of_week
0,<NA>,Photo,<NA>,<NA>,1613575431,304,637,False,1538731851,2625,2587,False,1449472125,True,0,0,0,1613575581,308728,2094195,123065,101538,0,0,0,61,1,1,0,0,15,17,2,2
1,<NA>,<NA>,<NA>,<NA>,1612886582,23310,2119,True,1246300047,120,209,False,1309436731,False,0,0,0,0,5302839,1935527,200136,7414,0,0,0,48,1,0,0,0,16,9,2,1
2,<NA>,<NA>,<NA>,<NA>,1614130621,38519,21564,False,1257360731,1032,1926,False,1533159537,True,0,0,0,1614164644,4558586,2178656,217822,26088,0,0,0,48,2,0,0,0,1,24,2,2
3,<NA>,Photo\tPhoto,<NA>,<NA>,1614184644,14106,1645,False,1223668013,265,335,False,1249692829,False,0,0,0,1614189647,5536323,1894129,150116,150980,0,0,0,19,1,2,0,0,16,24,2,2
4,8EC708C0073EDDE57F3BB17ECA262D9E\tCBFFDD847281...,Video,<NA>,<NA>,1613842378,2652,175,False,1562820796,406,794,False,1572957202,False,0,0,0,1613855637,1017697,1981260,127092,80160,2,0,0,45,2,0,0,1,17,20,2,5
